In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# Landmark indices untuk deteksi posisi kepala
NOSE_TIP = 1
CHIN = 175
LEFT_EYE_CORNER = 33
RIGHT_EYE_CORNER = 263
LEFT_MOUTH_CORNER = 61
RIGHT_MOUTH_CORNER = 291

class DrowsinessDetector:
    def __init__(self):
        self.head_down_threshold = 50  # derajat (lebih rendah untuk head down)
        self.head_tilt_threshold = 20  # derajat
        self.consecutive_frames = 30   # frames berturut-turut untuk konfirmasi
        
        self.head_down_counter = 0
        self.head_tilt_counter = 0
        self.drowsy_counter = 0
        
        self.alert_start_time = None
        self.is_drowsy = False
    
    def get_face_bbox(self, landmarks, img_w, img_h):
        """Menghitung bounding box wajah berdasarkan landmark"""
        try:
            x_coords = [landmark.x * img_w for landmark in landmarks]
            y_coords = [landmark.y * img_h for landmark in landmarks]
            
            if not x_coords or not y_coords:
                return None
            
            x_min = int(min(x_coords))
            y_min = int(min(y_coords))
            x_max = int(max(x_coords))
            y_max = int(max(y_coords))
            
            margin = 20
            x_min = max(0, x_min - margin)
            y_min = max(0, y_min - margin)
            x_max = min(img_w, x_max + margin)
            y_max = min(img_h, y_max + margin)
            
            return (x_min, y_min, x_max, y_max)
        except Exception as e:
            print(f"Error in get_face_bbox: {e}")
            return None

    def get_head_pose(self, landmarks, img_w, img_h):
        """Menghitung pose kepala menggunakan landmark wajah"""
        try:
            model_points = np.array([
                (0.0, 0.0, 0.0),             # Nose tip
                (0.0, -330.0, -65.0),        # Chin
                (-225.0, 170.0, -135.0),     # Left eye left corner
                (225.0, 170.0, -135.0),      # Right eye right corner
                (-150.0, -150.0, -125.0),    # Left Mouth corner
                (150.0, -150.0, -125.0)      # Right mouth corner
            ])
            
            image_points = np.array([
                (landmarks[NOSE_TIP].x * img_w, landmarks[NOSE_TIP].y * img_h),
                (landmarks[CHIN].x * img_w, landmarks[CHIN].y * img_h),
                (landmarks[LEFT_EYE_CORNER].x * img_w, landmarks[LEFT_EYE_CORNER].y * img_h),
                (landmarks[RIGHT_EYE_CORNER].x * img_w, landmarks[RIGHT_EYE_CORNER].y * img_h),
                (landmarks[LEFT_MOUTH_CORNER].x * img_w, landmarks[LEFT_MOUTH_CORNER].y * img_h),
                (landmarks[RIGHT_MOUTH_CORNER].x * img_w, landmarks[RIGHT_MOUTH_CORNER].y * img_h)
            ], dtype="double")
            
            focal_length = img_w
            center = (img_w/2, img_h/2)
            camera_matrix = np.array(
                [[focal_length, 0, center[0]],
                 [0, focal_length, center[1]],
                 [0, 0, 1]], dtype="double"
            )
            
            dist_coeffs = np.zeros((4,1))
            
            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )
            
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            angles = cv2.RQDecomp3x3(rotation_matrix)[0]
            
            return angles
        except Exception as e:
            print(f"Error in get_head_pose: {e}")
            return [0, 0, 0]
    
    def detect_drowsiness(self, landmarks, img_w, img_h):
        """Deteksi kantuk hanya berdasarkan posisi kepala"""
        try:
            # 1. Deteksi posisi kepala
            angles = self.get_head_pose(landmarks, img_w, img_h)
            pitch, yaw, roll = angles
            
            # 2. Logika deteksi kantuk
            drowsy_indicators = []
            
            # Kepala menunduk (pitch negatif menunjukkan kepala turun)
            if pitch < -self.head_down_threshold:
                self.head_down_counter += 1
                drowsy_indicators.append("HEAD_DOWN")
            else:
                self.head_down_counter = 0
            
            # Kepala miring berlebihan
            if abs(roll) > self.head_tilt_threshold:
                self.head_tilt_counter += 1
                drowsy_indicators.append("HEAD_TILT")
            else:
                self.head_tilt_counter = 0
            
            # Kombinasi indikator untuk menentukan kantuk
            is_drowsy = (
                self.head_down_counter >= self.consecutive_frames or
                self.head_tilt_counter >= self.consecutive_frames
            )
            
            if is_drowsy and not self.is_drowsy:
                self.alert_start_time = time.time()
                self.is_drowsy = True
            
            if not is_drowsy:
                self.is_drowsy = False
                self.alert_start_time = None
            
            return {
                'is_drowsy': is_drowsy,
                'pitch': pitch,
                'yaw': yaw,
                'roll': roll,
                'indicators': drowsy_indicators,
                'counters': {
                    'head_down': self.head_down_counter,
                    'head_tilt': self.head_tilt_counter,
                }
            }
        except Exception as e:
            print(f"Error in detect_drowsiness: {e}")
            return {
                'is_drowsy': False,
                'pitch': 0,
                'yaw': 0,
                'roll': 0,
                'indicators': [],
                'counters': {
                    'head_down': 0,
                    'head_tilt': 0,
                }
            }

# Implementasi dalam loop utama
detector = DrowsinessDetector()
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w = image.shape[:2]
                
                drowsy_result = detector.detect_drowsiness(
                    face_landmarks.landmark, w, h
                )
                
                bbox = detector.get_face_bbox(face_landmarks.landmark, w, h)
                
                if bbox is not None:
                    x_min, y_min, x_max, y_max = bbox
                    
                    bbox_color = (0, 0, 255) if drowsy_result['is_drowsy'] else (0, 255, 0)
                    bbox_thickness = 3 if drowsy_result['is_drowsy'] else 2
                    
                    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), bbox_color, bbox_thickness)
                    
                    label_text = "DROWSY" if drowsy_result['is_drowsy'] else "ALERT"
                    label_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    
                    cv2.rectangle(image, (x_min, y_min - 30), (x_min + label_size[0] + 10, y_min), bbox_color, -1)
                    cv2.putText(image, label_text, (x_min + 5, y_min - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                else:
                    status_text = "DROWSY" if drowsy_result['is_drowsy'] else "ALERT"
                    status_color = (0, 0, 255) if drowsy_result['is_drowsy'] else (0, 255, 0)
                    cv2.putText(image, f"Status: {status_text}", (10, 200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)
                
                info_y_start = 30
                cv2.putText(image, f"Pitch: {drowsy_result['pitch']:.1f}", (10, info_y_start), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(image, f"Yaw: {drowsy_result['yaw']:.1f}", (10, info_y_start + 25), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(image, f"Roll: {drowsy_result['roll']:.1f}", (10, info_y_start + 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                if drowsy_result['indicators']:
                    indicators_text = ", ".join(drowsy_result['indicators'])
                    cv2.putText(image, f"Indicators: {indicators_text}", (10, info_y_start + 75), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                
                if drowsy_result['is_drowsy']:
                    overlay = image.copy()
                    cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 255), -1)
                    image = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)
                    
                    cv2.putText(image, "DROWSINESS DETECTED!", (w//2 - 250, h//2), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

        cv2.imshow('Drowsiness Detection', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()